In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

dataset_dir = 'C:/Users/vsavelev/GITHUB/DS_projet/jan24_cds_mushrooms/data'

# Create ImageDataGenerator with validation split
datagen = ImageDataGenerator(rescale=1.0/255, validation_split=0.2)

train_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'  # Set as training data
)

validation_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'  # Set as validation data
)

print(f'Training samples: {train_generator.samples}')
print(f'Validation samples: {validation_generator.samples}')

Found 2199 images belonging to 2 classes.
Found 549 images belonging to 2 classes.
Training samples: 2199
Validation samples: 549


In [2]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models
import keras_tuner as kt

def build_model(hp):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False

    model = models.Sequential()
    model.add(base_model)
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dense(hp.Int('units', min_value=256, max_value=1024, step=256), activation='relu'))
    model.add(layers.Dropout(hp.Float('dropout', min_value=0.2, max_value=0.5, step=0.1)))
    model.add(layers.Dense(train_generator.num_classes, activation='softmax'))
    
    model.compile(optimizer=tf.keras.optimizers.Adam(hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model


In [3]:
#Set Up the Tuner

tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='my_dir',
    project_name='mushroom_classification'
)


In [4]:
tuner.search(train_generator, epochs=10, validation_data=validation_generator)

Trial 5 Complete [05h 32m 33s]
val_accuracy: 0.8870673775672913

Best val_accuracy So Far: 0.8870673775672913
Total elapsed time: 11h 14m 56s


In [5]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the dense layer is {best_hps.get('units')}.
The optimal learning rate for the optimizer is {best_hps.get('learning_rate')}.
""")

# Build the best model
model = tuner.hypermodel.build(best_hps)
model.summary()


The hyperparameter search is complete. The optimal number of units in the dense layer is 768.
The optimal learning rate for the optimizer is 0.001.



Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ ?                      │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ ?                      │   0 (unbuilt) │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,587,712 (89.98 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 23,587,712 (89.98 MB)

In [6]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10
)

#This specifies the number of complete passes through the training dataset. Here, the model will train for 10 epochs.

Epoch 1/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 164s 2s/step - accuracy: 0.8856 - loss: 0.4458 - val_accuracy: 0.8860 - val_loss: 0.3645
Epoch 2/10
 1/68 ━━━━━━━━━━━━━━━━━━━━ 1:57 2s/step - accuracy: 0.9062 - loss: 0.3521

c:\Users\vsavelev\AppData\Local\anaconda3\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


68/68 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.9062 - loss: 0.3521 - val_accuracy: 1.0000 - val_loss: 0.0784
Epoch 3/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 153s 2s/step - accuracy: 0.8923 - loss: 0.3465 - val_accuracy: 0.8879 - val_loss: 0.3535
Epoch 4/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8750 - loss: 0.4432 - val_accuracy: 0.8000 - val_loss: 0.5806
Epoch 5/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 151s 2s/step - accuracy: 0.8817 - loss: 0.3742 - val_accuracy: 0.8860 - val_loss: 0.3608
Epoch 6/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9375 - loss: 0.2319 - val_accuracy: 1.0000 - val_loss: 0.0935
Epoch 7/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 156s 2s/step - accuracy: 0.8882 - loss: 0.3576 - val_accuracy: 0.8860 - val_loss: 0.3543
Epoch 8/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.8438 - loss: 0.4161 - val_accuracy: 1.0000 - val_loss: 0.1267
Epoch 9/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 154s 2s/step - accuracy: 0.8969 - loss: 0.3322 - val_accuracy: 0.8860 - val_loss: 0.374

In [7]:
loss, accuracy = model.evaluate(validation_generator)
print(f'Validation loss: {loss}')
print(f'Validation accuracy: {accuracy}')


18/18 ━━━━━━━━━━━━━━━━━━━━ 30s 2s/step - accuracy: 0.8850 - loss: 0.3732
Validation loss: 0.36864525079727173
Validation accuracy: 0.8870673775672913
